## Feature Engineering

#### Importamos las librerias necesarias para realizar las funciones

In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

%load_ext autoreload
%autoreload 2
import herramientas

import warnings
warnings.filterwarnings("ignore")

Leemos y guardamos en dataframes , las base de datos limpias 

In [4]:
df_reviews=pd.read_csv(r'data\user_review_limpio.csv')
df_items=pd.read_csv(r'data\user_items_limpio.csv')
df_games=pd.read_csv(r'data\steam_games_limpio.csv')

Verificamos nulos de las mismas 

In [5]:
herramientas.verifica_tipo_y_nulos(df_items)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,item_id,[<class 'int'>],100.0,0.0,0
1,item_name,[<class 'str'>],100.0,0.0,0
2,playtime_forever,[<class 'int'>],100.0,0.0,0
3,user_id,[<class 'str'>],100.0,0.0,0
4,items_count,[<class 'int'>],100.0,0.0,0
5,steam_id,[<class 'int'>],100.0,0.0,0


In [6]:
herramientas.verifica_tipo_y_nulos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'str'>],100.00,0.00,0
1,price,[<class 'float'>],100.00,0.00,0
2,early_access,[<class 'bool'>],100.00,0.00,0
3,id,[<class 'int'>],100.00,0.00,0
4,release_anio,[<class 'str'>],100.00,0.00,0
5,publisher,"[<class 'str'>, <class 'float'>]",99.93,0.07,47
6,title,[<class 'str'>],100.00,0.00,0
7,developer,[<class 'str'>],100.00,0.00,0


In [7]:
herramientas.verifica_tipo_y_nulos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,reviews_user_id,[<class 'str'>],100.00,0.00,0
1,reviews_item_id,[<class 'int'>],100.00,0.00,0
2,reviews_helpful,[<class 'str'>],100.00,0.00,0
3,reviews_recommend,[<class 'bool'>],100.00,0.00,0
4,reviews_review,"[<class 'str'>, <class 'float'>]",99.95,0.05,30
5,reviews_date,[<class 'str'>],100.00,0.00,0
6,sentiment_analysis,[<class 'int'>],100.00,0.00,0
7,rating,[<class 'float'>],85.46,14.54,8497


Las columnas con presencias de nulos las limpiaremos en caso de utilizarlas , ademas posteriormente crearemos tablas por funciones optimizadas para luego convertirlas en .parquet para poder deployar en Render

- **Funcion 'PlayTimeGenre(genero)'**

Renombre las columas id por item_id en el df_games y le sacamos el prefijo 'reviews' del df_reviews, para posteriormente relacionar dicho dataframe con df_reviews

In [8]:
df_games.rename(columns={'id':'item_id'}, inplace= True)
df_reviews.rename(columns={'reviews_item_id':'item_id','reviews_user_id':'user_id'}, inplace= True)

Realizo el merge de los dataframes , y ademas le indicamos que nos traiga los distintos de 0 de la columna playtime_forever

In [9]:
df_merge=df_games[['genres','release_anio','item_id']]
df_merge1=df_items[['playtime_forever', 'item_id']]

df_merge1=df_merge1.loc[df_merge1['playtime_forever'] != 0]

df_merge2=pd.merge(df_merge,df_merge1,on='item_id')
df_merge2


,genres,release_anio,item_id,playtime_forever
0,Action,1997,282010,5
1,Action,1997,282010,13
2,Action,1997,282010,110
3,Action,1997,282010,13
4,Action,1997,282010,8
...,...,...,...,...
6845639,Action,2004,80,203
6845640,Action,2004,80,244
6845641,Action,2004,80,7
6845642,Action,2004,80,9


Realizo la agrupacion por genres y release_anio.
Luego suma todos los valores en la columna ‘playtime_forever’ para cada grupo y reinicio el indice

In [10]:
df_grouped = df_merge2.groupby(['genres', 'release_anio'])['playtime_forever'].sum().reset_index()
df_grouped

,genres,release_anio,playtime_forever
0,Action,1983,3473
1,Action,1984,384
2,Action,1988,16001
3,Action,1989,607
4,Action,1990,18335
...,...,...,...
348,Web Publishing,2013,333678
349,Web Publishing,2014,33641
350,Web Publishing,2015,348673
351,Web Publishing,2016,136


Por ultimo guardo mi tabla en un archivo .parquet posteriormente avanzar con el proceso de deploy en render

In [11]:
#Guarda el DataFrame en el archivo PARQUET
df_grouped.to_parquet('data/df_playtime.parquet')

print(f'Se guardó el archivo {df_grouped}')

Se guardó el archivo              genres release_anio  playtime_forever
0            Action         1983              3473
1            Action         1984               384
2            Action         1988             16001
3            Action         1989               607
4            Action         1990             18335
..              ...          ...               ...
348  Web Publishing         2013            333678
349  Web Publishing         2014             33641
350  Web Publishing         2015            348673
351  Web Publishing         2016               136
352  Web Publishing         2017              9382

[353 rows x 3 columns]


Leemos nuestro archivo parquet con un nuevo dataframe

In [12]:
df_play=pd.read_parquet('data/df_playtime.parquet')

Verificamos las dimenciones del mismo 

In [13]:
df_play

,genres,release_anio,playtime_forever
0,Action,1983,3473
1,Action,1984,384
2,Action,1988,16001
3,Action,1989,607
4,Action,1990,18335
...,...,...,...
348,Web Publishing,2013,333678
349,Web Publishing,2014,33641
350,Web Publishing,2015,348673
351,Web Publishing,2016,136


Creo mi funcion PlayTimeGenre( genero : str ): La cual debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [14]:
def PlayTimeGenre(genero):
      
    # Filtrar por el género especificado
    df_genre = df_play[df_play['genres'] == genero]
    
    # Si no hay datos para el género especificado, retorna un mensaje
    if df_genre.empty:
        return f"No hay datos para el género '{genero}'"
    
    # Agrupar por año y calcular las horas jugadas sumando los valores
    grouped = df_genre.groupby('release_anio')['playtime_forever'].sum()
    
    # Encontrar el año con más horas jugadas
    max_playtime_year = grouped.idxmax()
        
    # Retornar el resultado como un diccionario
    return {"Año de lanzamiento con más horas jugadas para Género {}".format(genero): max_playtime_year}

In [15]:
PlayTimeGenre('Action')

{'Año de lanzamiento con más horas jugadas para Género Action': '2012'}

Finalmente comprobamos el funcionamiento de la funcion y seguimos con las funciones siguientes 

- **Funcion 'UserForGenre(genero)'**

Realizamos la creacion de mis dataframes necesarios para extraer la tabla necesaria para la consulta y posteriormente realizo el merge de las mismas para crear un dataframe

In [16]:
df_merge_g=df_games[['genres','item_id','release_anio',]]
df_merge_g1=df_items[['playtime_forever','item_id','user_id']]
df_merge_g2=pd.merge(df_merge_g1,df_merge_g,on= 'item_id')
df_merge_g2

,playtime_forever,item_id,user_id,genres,release_anio
0,6,10,76561197970982479,Action,2000
1,0,10,js41637,Action,2000
2,0,10,Riot-Punch,Action,2000
3,93,10,doctr,Action,2000
4,108,10,corrupted_soul,Action,2000
...,...,...,...,...,...
9877250,164,354280,76561198107283457,Indie,2016
9877251,164,354280,76561198107283457,Simulation,2016
9877252,0,433920,inven,Adventure,2016
9877253,0,433920,inven,Indie,2016


Creo un dataframe que me traiga los valores distintos de 0 

In [17]:
df_merge_g2_clean=df_merge_g2.loc[df_merge_g2['playtime_forever'] != 0]
df_merge_g2_clean

,playtime_forever,item_id,user_id,genres,release_anio
0,6,10,76561197970982479,Action,2000
3,93,10,doctr,Action,2000
4,108,10,corrupted_soul,Action,2000
7,328,10,WeiEDKrSat,Action,2000
8,6275,10,death-hunter,Action,2000
...,...,...,...,...,...
9877247,51,375450,76561198081168593,Strategy,2016
9877248,164,354280,76561198107283457,Action,2016
9877249,164,354280,76561198107283457,Casual,2016
9877250,164,354280,76561198107283457,Indie,2016


Realizo la agrupoacion por las columas 'genres', 'user_id', 'release_anio' y sumo los valores en la columna ‘playtime_forever’, ademas de reestablecer el indice del dataframe

In [18]:
df_merge_g2_group=df_merge_g2_clean.groupby(['genres', 'user_id', 'release_anio'])['playtime_forever'].sum().reset_index()
df_merge_g2_group

,genres,user_id,release_anio,playtime_forever
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363
...,...,...,...,...
2893382,Web Publishing,zepavil,2015,9010
2893383,Web Publishing,zeshirky,2007,1
2893384,Web Publishing,zevlupine,2012,4
2893385,Web Publishing,zilaman,2013,9


Renombro el dataframe para guardar los cambios

In [19]:
df_useforgenre=df_merge_g2_group

In [20]:
#Guarda el DataFrame en el archivo PARQUET
df_useforgenre.to_parquet('data/df_useforgenre.parquet')

print(f'Se guardó el archivo {df_useforgenre}')

Se guardó el archivo                  genres    user_id release_anio  playtime_forever
0                Action    --000--         2009              5329
1                Action    --000--         2010                22
2                Action    --000--         2011              6522
3                Action    --000--         2012            109346
4                Action    --000--         2013               363
...                 ...        ...          ...               ...
2893382  Web Publishing    zepavil         2015              9010
2893383  Web Publishing   zeshirky         2007                 1
2893384  Web Publishing  zevlupine         2012                 4
2893385  Web Publishing    zilaman         2013                 9
2893386  Web Publishing  zipper559         2013                 3

[2893387 rows x 4 columns]


In [21]:
df_useforgenre=pd.read_parquet('data/df_useforgenre.parquet')
df_useforgenre

,genres,user_id,release_anio,playtime_forever
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363
...,...,...,...,...
2893382,Web Publishing,zepavil,2015,9010
2893383,Web Publishing,zeshirky,2007,1
2893384,Web Publishing,zevlupine,2012,4
2893385,Web Publishing,zilaman,2013,9


In [22]:
herramientas.verifica_tipo_y_nulos(df_useforgenre)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'str'>],100.0,0.0,0
1,user_id,[<class 'str'>],100.0,0.0,0
2,release_anio,[<class 'str'>],100.0,0.0,0
3,playtime_forever,[<class 'int'>],100.0,0.0,0


In [23]:
def UserForGenre(genero):
        
    # Filtrar por el género especificado
    df_genre = df_useforgenre[df_useforgenre['genres'] == genero]
    
    # Si no hay datos para el género especificado, retorna un mensaje
    if df_genre.empty:
        return f"No hay datos para el género '{genero}'"
    
    # Agrupar por usuario y género y calcular las horas jugadas sumando los valores
    grouped = df_genre.groupby(['user_id'])['playtime_forever'].sum()
    
    # Encontrar el usuario con más horas jugadas
    max_playtime_user = grouped.idxmax()
        
    # Agrupar por año y calcular las horas jugadas sumando los valores
    grouped_by_year = df_genre.groupby('release_anio')['playtime_forever'].sum()
    
    # Crear lista de acumulación de horas jugadas por año
    acumulacion_horas = [{'Año': year, 'Horas': hours} for year, hours in grouped_by_year.items()]
    
    # Retornar el resultado como un diccionario
    return {"Usuario con más horas jugadas para Género {}".format(genero): max_playtime_user, "Horas jugadas": acumulacion_horas}


In [24]:
UserForGenre('Action')

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': '1983', 'Horas': 3473},
  {'Año': '1984', 'Horas': 384},
  {'Año': '1988', 'Horas': 16001},
  {'Año': '1989', 'Horas': 607},
  {'Año': '1990', 'Horas': 18335},
  {'Año': '1991', 'Horas': 2502},
  {'Año': '1992', 'Horas': 1925},
  {'Año': '1993', 'Horas': 24623},
  {'Año': '1994', 'Horas': 120177},
  {'Año': '1995', 'Horas': 216144},
  {'Año': '1996', 'Horas': 67844},
  {'Año': '1997', 'Horas': 359836},
  {'Año': '1998', 'Horas': 2899116},
  {'Año': '1999', 'Horas': 2715157},
  {'Año': '2000', 'Horas': 18357855},
  {'Año': '2001', 'Horas': 1326310},
  {'Año': '2002', 'Horas': 732102},
  {'Año': '2003', 'Horas': 11893403},
  {'Año': '2004', 'Horas': 129693956},
  {'Año': '2005', 'Horas': 12264943},
  {'Año': '2006', 'Horas': 10772811},
  {'Año': '2007', 'Horas': 17530208},
  {'Año': '2008', 'Horas': 33539669},
  {'Año': '2009', 'Horas': 163799096},
  {'Año': '2010', 'Horas': 180087364},
  {'Año': '

- **Funcion UsersRecommend(año)**

In [25]:
herramientas.verifica_tipo_y_nulos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.00,0.00,0
1,item_id,[<class 'int'>],100.00,0.00,0
2,reviews_helpful,[<class 'str'>],100.00,0.00,0
3,reviews_recommend,[<class 'bool'>],100.00,0.00,0
4,reviews_review,"[<class 'str'>, <class 'float'>]",99.95,0.05,30
5,reviews_date,[<class 'str'>],100.00,0.00,0
6,sentiment_analysis,[<class 'int'>],100.00,0.00,0
7,rating,[<class 'float'>],85.46,14.54,8497


In [26]:
herramientas.verifica_tipo_y_nulos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'str'>],100.00,0.00,0
1,price,[<class 'float'>],100.00,0.00,0
2,early_access,[<class 'bool'>],100.00,0.00,0
3,item_id,[<class 'int'>],100.00,0.00,0
4,release_anio,[<class 'str'>],100.00,0.00,0
5,publisher,"[<class 'str'>, <class 'float'>]",99.93,0.07,47
6,title,[<class 'str'>],100.00,0.00,0
7,developer,[<class 'str'>],100.00,0.00,0


Verifico la existencia de valores = 2 en la column a 'rating'

In [27]:
df_reviews['rating'] == 2

0        True
1        True
2        True
3        True
4        True
         ... 
58425    True
58426    True
58427    True
58428    True
58429    True
Name: rating, Length: 58430, dtype: bool

In [28]:
# Convierte la columna 'reviews_date' a datetime
df_reviews['reviews_date'] = pd.to_datetime(df_reviews['reviews_date'], errors='coerce')

# Elimina las filas donde 'reviews_date' es NaT
df_reviews = df_reviews.dropna(subset=['reviews_date'])

# Extrae el año de 'reviews_date' a una nueva columna 'year'
df_reviews['year'] = df_reviews['reviews_date'].dt.year.astype(int)

# Elimina la columna 'reviews_date'
df_reviews = df_reviews.drop('reviews_date', axis=1)

df_reviews


,user_id,item_id,reviews_helpful,reviews_recommend,reviews_review,sentiment_analysis,rating,year
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1,2.0,2011
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2,2.0,2011
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1,2.0,2011
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1,2.0,2014
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,1,2.0,2013
...,...,...,...,...,...,...,...,...
58377,wayfeng,730,1 of 1 people (100%) found this review helpful,True,its FUNNNNNNNN,1,2.0,2015
58380,76561198251004808,253980,No ratings yet,True,Awesome fantasy game if you don't mind the gra...,2,2.0,2015
58390,72947282842,730,No ratings yet,True,Prettyy Mad Game,0,NaN,2015
58392,ApxLGhost,730,No ratings yet,True,AMAZING GAME 10/10,1,2.0,2015


In [29]:
df_reviews1 = df_reviews[df_reviews['year'].duplicated()]
df_reviews1

,user_id,item_id,reviews_helpful,reviews_recommend,reviews_review,sentiment_analysis,rating,year
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2,2.0,2011
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1,2.0,2011
5,js41637,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,1,2.0,2013
8,evcentric,237930,No ratings yet,True,"Elegant integration of gameplay, story, world ...",2,2.0,2014
9,evcentric,263360,No ratings yet,True,"Random drops and random quests, with stat poin...",0,NaN,2014
...,...,...,...,...,...,...,...,...
58377,wayfeng,730,1 of 1 people (100%) found this review helpful,True,its FUNNNNNNNN,1,2.0,2015
58380,76561198251004808,253980,No ratings yet,True,Awesome fantasy game if you don't mind the gra...,2,2.0,2015
58390,72947282842,730,No ratings yet,True,Prettyy Mad Game,0,NaN,2015
58392,ApxLGhost,730,No ratings yet,True,AMAZING GAME 10/10,1,2.0,2015


In [30]:

herramientas.verifica_tipo_y_nulos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.00,0.00,0
1,item_id,[<class 'int'>],100.00,0.00,0
2,reviews_helpful,[<class 'str'>],100.00,0.00,0
3,reviews_recommend,[<class 'bool'>],100.00,0.00,0
4,reviews_review,"[<class 'str'>, <class 'float'>]",99.94,0.06,27
5,sentiment_analysis,[<class 'int'>],100.00,0.00,0
6,rating,[<class 'float'>],86.45,13.55,6570
7,year,[<class 'int'>],100.00,0.00,0


In [31]:
df_merge_r2 = pd.merge(df_games[['title','item_id']], df_reviews[['rating','item_id','year']], on='item_id')
df_merge_reco1 = df_merge_r2.dropna(subset=['rating'])
df_merge_reco1 = df_merge_reco1[df_merge_reco1['rating'] == 2]


In [32]:
df_merge_reco1

,title,item_id,rating,year
0,Half-Life,70,2.0,2015
1,Half-Life,70,2.0,2011
2,Half-Life,70,2.0,2014
3,Half-Life,70,2.0,2013
4,Half-Life,70,2.0,2013
...,...,...,...,...
102143,Counter-Strike: Condition Zero,80,2.0,2011
102144,Counter-Strike: Condition Zero,80,2.0,2014
102147,Counter-Strike: Condition Zero,80,2.0,2013
102148,Counter-Strike: Condition Zero,80,2.0,2015


In [33]:
#Agrupar por año y título, contar la frecuencia y clasificar en orden descendente
df_grouped = df_merge_reco1.groupby(['year', 'title']).size().reset_index(name='count').sort_values(by=['year', 'count'], ascending=[True, False])

#Crear una nueva columna 'rank' dentro de cada grupo
df_grouped['rank'] = df_grouped.groupby('year')['count'].rank(ascending=False, method='first')

#Filtrar los tres títulos principales por año
df_top3 = df_grouped[df_grouped['rank'] <= 3]
df_top3

#Puedes seleccionar las columnas que desees en el nuevo DataFrame
result_df = df_top3[['year', 'title', 'rank']]
result_df

,year,title,rank
28,2010,Team Fortress 2,1.0
11,2010,Garry's Mod,2.0
12,2010,Killing Floor,3.0
168,2011,Team Fortress 2,1.0
169,2011,Terraria,2.0
138,2011,Portal 2,3.0
371,2012,Team Fortress 2,1.0
373,2012,Terraria,2.0
330,2012,Realm of the Mad God,3.0
881,2013,Team Fortress 2,1.0


In [34]:
df_recom=result_df

In [35]:
def UsersRecommend(año: int):
    # Filtra el DataFrame df_recom por el año proporcionado y lo guarda en top3_by_year
    top3_by_year = df_recom[df_recom['year'] == año]

    # Inicializa una lista vacía para almacenar los resultados
    resultado = []
    
    # Itera sobre las filas del DataFrame top3_by_year
    for index, row in top3_by_year.iterrows():
        # Obtiene el puesto, el título y el año de la fila actual
        puesto = row['rank']
        titulo = row['title']
        año = int(row['year'])
        
        # Agrega un diccionario a la lista de resultados con la estructura {f"Puesto {puesto}": f"{titulo}"}
        resultado.append({f"Puesto {puesto}": f"{titulo}"})
    
    # Devuelve la lista de resultados
    return resultado


In [36]:
UsersRecommend(2015)

[{'Puesto 1.0': 'Counter-Strike: Global Offensive'},
 {'Puesto 2.0': 'Unturned'},
 {'Puesto 3.0': 'Team Fortress 2'}]

In [37]:
#Guarda el DataFrame en el archivo PARQUET
df_recom.to_parquet('data/df_recom.parquet')

print(f'Se guardó el archivo {df_recom}')

Se guardó el archivo       year                             title  rank
28    2010                   Team Fortress 2   1.0
11    2010                       Garry's Mod   2.0
12    2010                     Killing Floor   3.0
168   2011                   Team Fortress 2   1.0
169   2011                          Terraria   2.0
138   2011                          Portal 2   3.0
371   2012                   Team Fortress 2   1.0
373   2012                          Terraria   2.0
330   2012              Realm of the Mad God   3.0
881   2013                   Team Fortress 2   1.0
615   2013                       Garry's Mod   2.0
882   2013                          Terraria   3.0
1969  2014                   Team Fortress 2   1.0
2105  2014                          Unturned   2.0
1804  2014                              Rust   3.0
2471  2015  Counter-Strike: Global Offensive   1.0
3650  2015                          Unturned   2.0
3469  2015                   Team Fortress 2   3.0


In [38]:
herramientas.verifica_tipo_y_nulos(df_merge_reco1)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,title,[<class 'str'>],100.0,0.0,0
1,item_id,[<class 'int'>],100.0,0.0,0
2,rating,[<class 'float'>],100.0,0.0,0
3,year,[<class 'int'>],100.0,0.0,0


Funcion **UsersWorstDeveloper(año)**

In [39]:
# Crea un nuevo DataFrame df_games_w que contiene solo las columnas 'developer' e 'item_id' de df_games
df_games_w=df_games[['developer','item_id']]

# Crea otro DataFrame df_reviews_w que contiene solo las columnas 'rating', 'item_id' y 'year' de df_reviews
df_reviews_w=df_reviews[['rating','item_id','year']]

# Une df_games_w y df_reviews_w en un nuevo DataFrame df_merge basándose en la columna 'item_id'
df_merge=pd.merge(df_games_w,df_reviews_w, on='item_id')

# Elimina las filas de df_merge donde la columna 'rating' es NaN y guarda el DataFrame resultante en df_merge_clean
df_merge_clean=df_merge.dropna(subset='rating')

# Filtra df_merge_clean para incluir solo las filas donde la columna 'rating' es igual a 1 y guarda el DataFrame resultante en df_merge_rating
df_merge_rating=df_merge_clean[df_merge_clean['rating'] == 1]

# Agrupa df_merge_rating por 'year' y 'developer', cuenta el número de filas en cada grupo, ordena los grupos por 'year' y 'count' en orden ascendente y guarda el DataFrame resultante en df_merge_w
df_merge_w = df_merge_rating.groupby(['year', 'developer']).size().reset_index(name='count').sort_values(by=['year', 'count'], ascending=[True, True])

# Crea una nueva columna 'rank' en df_merge_w que contiene el rango de 'count' dentro de cada grupo de 'year'
df_merge_w ['rank'] = df_merge_w .groupby('year')['count'].rank(ascending=False, method='first')

# Filtra df_merge_w para incluir solo las filas donde la columna 'rank' es menor o igual a 3 y guarda el DataFrame resultante en df_worst
df_worst = df_merge_w[df_merge_w['rank'] <= 3]

# Filtra df_worst para excluir las filas donde la columna 'year' es igual a 'Dato no disponible' y guarda el DataFrame resultante en df_worst_1
df_worst_1= df_worst[df_worst['year'] != 'Dato no disponible']

# Convierte la columna 'rank' en df_worst_1 a enteros
df_worst_1['rank']=df_worst_1['rank'].astype(int)
df_worst_1


,year,developer,count,rank
0,2011,1C Company,1,3
1,2011,Broken Rules,2,1
2,2011,Valve,2,2
3,2012,NeoCoreGames,1,2
5,2012,Volition,1,3
4,2012,Valve,2,1
27,2013,Facepunch Studios,8,2
51,2013,Valve,8,3
41,2013,Phosphor Games Studio,10,1
118,2014,Facepunch Studios,54,3


In [40]:
herramientas.verifica_tipo_y_nulos(df_worst_1)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,year,[<class 'int'>],100.0,0.0,0
1,developer,[<class 'str'>],100.0,0.0,0
2,count,[<class 'int'>],100.0,0.0,0
3,rank,[<class 'int'>],100.0,0.0,0


In [41]:
#Guarda el DataFrame en el archivo PARQUET
df_worst_1.to_parquet('data/df_worst.parquet')

print(f'Se guardó el archivo {df_worst_1}')

Se guardó el archivo      year              developer  count  rank
0    2011             1C Company      1     3
1    2011           Broken Rules      2     1
2    2011                  Valve      2     2
3    2012           NeoCoreGames      1     2
5    2012               Volition      1     3
4    2012                  Valve      2     1
27   2013      Facepunch Studios      8     2
51   2013                  Valve      8     3
41   2013  Phosphor Games Studio     10     1
118  2014      Facepunch Studios     54     3
224  2014   Trek Industries, Inc     56     2
65   2014    Bohemia Interactive     59     1
413  2015  Smartly Dressed Games     65     3
264  2015    Bohemia Interactive     77     2
308  2015      Facepunch Studios    104     1


In [42]:
df_worst=pd.read_parquet('data/df_worst.parquet')

In [43]:
def UsersWorstDeveloper( año : int ):
    # Filtrar el DataFrame df_developer por el año proporcionado
    developer_by_year = df_worst_1[df_worst_1['year'] == año]

    # Obtener el top 3 de desarrolladoras con juegos MENOS recomendados y sus valores según rank
    top3_worst_developer = developer_by_year.sort_values(by='rank', ascending=True).head(3)

    # Formatear el resultado como lista de diccionarios
    result = [{"Puesto {}: {}".format(rank, developer)} for rank, developer in zip(top3_worst_developer['rank'], top3_worst_developer['developer'])]

    return result


In [44]:
UsersWorstDeveloper(2011)

[{'Puesto 1: Broken Rules'}, {'Puesto 2: Valve'}, {'Puesto 3: 1C Company'}]

- **Funcion: sentiment_analysis(año)**

In [45]:
# Selecciona las columnas 'developer' e 'item_id' del dataframe df_games y las guarda en un nuevo dataframe df_games_d
df_games_d=df_games[['developer','item_id']]

# Selecciona las columnas 'item_id', 'reviews_recommend' y 'sentiment_analysis' del dataframe df_reviews y las guarda en un nuevo dataframe df_reviews_s
df_reviews_s=df_reviews[['item_id','reviews_recommend','sentiment_analysis']]

# Une los dataframes df_games_d y df_reviews_s en base a la columna 'item_id'
df_merge=pd.merge(df_games_d,df_reviews_s, on='item_id')

# Agrupa el dataframe unido por las columnas 'developer' y 'sentiment_analysis', cuenta el número de 'reviews_recommend' para cada grupo, y resetea el índice del dataframe
df_merge_group=df_merge.groupby(['developer', 'sentiment_analysis'])['reviews_recommend'].count().reset_index() 

# Renombra la columna 'reviews_recommend' a 'reviews_recommend_count' en el dataframe df_merge_group
df_merge_group = df_merge_group.rename(columns={'reviews_recommend': 'reviews_recommend_count'})

# Guarda el dataframe df_merge_group en una nueva variable df_senti
df_senti=df_merge_group

# Muestra el dataframe df_senti
df_senti


,developer,sentiment_analysis,reviews_recommend_count
0,07th Expansion,1,2
1,"10th Art Studio,Adventure Productions",1,2
2,"10th Art Studio,Adventure Productions",2,2
3,11 bit studios,0,21
4,11 bit studios,1,108
...,...,...,...
3065,tobyfox,2,60
3066,varlamov5264,1,4
3067,vidiludi games and entertainment,2,5
3068,webgames,1,3


In [46]:
herramientas.verifica_tipo_y_nulos(df_senti)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,developer,[<class 'str'>],100.0,0.0,0
1,sentiment_analysis,[<class 'int'>],100.0,0.0,0
2,reviews_recommend_count,[<class 'int'>],100.0,0.0,0


In [47]:
#Guarda el DataFrame en el archivo PARQUET
df_senti.to_parquet('data/df_senti.parquet')

print(f'Se guardó el archivo {df_senti}')

Se guardó el archivo                                   developer  sentiment_analysis  \
0                            07th Expansion                   1   
1     10th Art Studio,Adventure Productions                   1   
2     10th Art Studio,Adventure Productions                   2   
3                            11 bit studios                   0   
4                            11 bit studios                   1   
...                                     ...                 ...   
3065                                tobyfox                   2   
3066                           varlamov5264                   1   
3067       vidiludi games and entertainment                   2   
3068                               webgames                   1   
3069                            高考恋爱委员会,橘子班                   1   

      reviews_recommend_count  
0                           2  
1                           2  
2                           2  
3                          21  
4             

In [48]:
df_senti=pd.read_parquet('data/df_senti.parquet')
df_senti

,developer,sentiment_analysis,reviews_recommend_count
0,07th Expansion,1,2
1,"10th Art Studio,Adventure Productions",1,2
2,"10th Art Studio,Adventure Productions",2,2
3,11 bit studios,0,21
4,11 bit studios,1,108
...,...,...,...
3065,tobyfox,2,60
3066,varlamov5264,1,4
3067,vidiludi games and entertainment,2,5
3068,webgames,1,3


La función sentiment_analysis toma como entrada un nombre de empresa desarrolladora de juegos y devuelve un diccionario que contiene la cantidad de reseñas negativas, neutrales y positivas para esa empresa.

In [49]:
def sentiment_analysis(empresa_desarrolladora: str): 
    # Filtra el DataFrame df_senti por la empresa desarrolladora proporcionada y lo guarda en developer_df
    developer_df = df_senti[df_senti['developer'] == empresa_desarrolladora]

    # Crea un diccionario result con la estructura {empresa_desarrolladora: {'Negative': 0, 'Neutral': 0, 'Positive': 0}}
    result = {empresa_desarrolladora: {'Negative': 0, 'Neutral': 0, 'Positive': 0}}

    # Itera sobre las columnas 'sentiment_analysis' y 'reviews_recommend_count' del DataFrame developer_df
    for sentiment, count in zip(developer_df['sentiment_analysis'], developer_df['reviews_recommend_count']):
        # Crea un mapeo entre los valores numéricos de la columna 'sentiment_analysis' y las categorías de sentimiento correspondientes
        sentiment_mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        
        # Usa el mapeo para obtener la categoría de sentimiento correspondiente al valor numérico
        sentiment_category = sentiment_mapping[sentiment]
        
        # Suma la cantidad de reseñas (count) a la categoría de sentimiento correspondiente en el diccionario result
        result[empresa_desarrolladora][sentiment_category] += count

    # Devuelve el diccionario result
    return result


In [50]:
sentiment_analysis('Valve')

{'Valve': {'Negative': 834, 'Neutral': 6953, 'Positive': 4453}}